# 2. 파이썬으로 데이터 주무르기, pandas
**pandas를 활용해서 데이터프레임을 다뤄봅시다.**

1. Pandas 시작하기
    - prerequisite : Table
    - pandas import하기
   
2. Pandas로 1차원 데이터 다루기 - Series 
    - Series 선언하기
    - Series vs ndarray
    - Series vs dict
    - Series에 이름 붙이기
3. Pandas로 2차원 데이터 다루기 - dataframe
    - dataframe 선언하기
    - from csv to dataframe
    - dataframe 자료 접근하기

[수업에 사용된 covid 데이터](https://www.kaggle.com/imdevskp/corona-virus-report)

## I. pandas 시작하기

In [1]:
import pandas as pd

## II. pandas로 1차원 데이터 다루기 - Series

In [2]:
s = pd.Series([1,4,8,23,65])
s

0     1
1     4
2     8
3    23
4    65
dtype: int64

In [3]:
t = pd.Series({'one' : 1, 'two' : 2, 'three' : 3})
t

one      1
two      2
three    3
dtype: int64

#### Series + Numpy
- Series 는 ndarray와 유사하다

In [4]:
t[1]

2

In [5]:
t[1:3]

two      2
three    3
dtype: int64

In [6]:
s[s > s.median()] # 자기 자신의 중간값보다 큰 값들만 가지고 와라

3    23
4    65
dtype: int64

In [7]:
s[[3,1,4]]

3    23
1     4
4    65
dtype: int64

In [8]:
import numpy as np
np.exp(s)

0    2.718282e+00
1    5.459815e+01
2    2.980958e+03
3    9.744803e+09
4    1.694889e+28
dtype: float64

In [9]:
s.dtype

dtype('int64')

#### Series + dict

In [10]:
t

one      1
two      2
three    3
dtype: int64

In [11]:
t['one']

1

In [12]:
# Series에 값 추가
t['six'] = 6
t

one      1
two      2
three    3
six      6
dtype: int64

In [13]:
'six' in t

True

In [14]:
'seven' in t

False

In [15]:
t.get('seven',0)

0

#### Series 에 이름 붙이기

In [16]:
s = pd.Series(np.random.randn(5), name = 'random_nums')
s

0    0.075009
1   -1.703583
2   -0.083103
3   -0.173612
4    1.175945
Name: random_nums, dtype: float64

In [17]:
s.name = '임의의 난수'
s

0    0.075009
1   -1.703583
2   -0.083103
3   -0.173612
4    1.175945
Name: 임의의 난수, dtype: float64

## III. Pandas로 2차원 데이터 다루기 - dataframe
- 2-D 라벨 Table

In [18]:
d = {'height' : [1,2,3,4], 'weight' : [30,40,50,60]}
df = pd.DataFrame(d)
df

,height,weight
0,1,30
1,2,40
2,3,50
3,4,60


In [19]:
# 각 col의 데이터 형식을 볼 수 있음
df.dtypes

height    int64
weight    int64
dtype: object

##### csv 파일 -> dataframe
- csv 파일을 DataFrame으로 변경해줄 수 있다.
- .read_csv() 를 이용


In [20]:
covid = pd.read_csv('/content/country_wise_latest.csv')
covid

,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region
0,Afghanistan,36263,1269,25198,9796,106,10,18,3.50,69.49,5.04,35526,737,2.07,Eastern Mediterranean
1,Albania,4880,144,2745,1991,117,6,63,2.95,56.25,5.25,4171,709,17.00,Europe
2,Algeria,27973,1163,18837,7973,616,8,749,4.16,67.34,6.17,23691,4282,18.07,Africa
3,Andorra,907,52,803,52,10,0,0,5.73,88.53,6.48,884,23,2.60,Europe
4,Angola,950,41,242,667,18,1,0,4.32,25.47,16.94,749,201,26.84,Africa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,West Bank and Gaza,10621,78,3752,6791,152,2,0,0.73,35.33,2.08,8916,1705,19.12,Eastern Mediterranean
183,Western Sahara,10,1,8,1,0,0,0,10.00,80.00,12.50,10,0,0.00,Africa
184,Yemen,1691,483,833,375,10,4,36,28.56,49.26,57.98,1619,72,4.45,Eastern Mediterranean
185,Zambia,4552,140,2815,1597,71,1,465,3.08,61.84,4.97,3326,1226,36.86,Africa


#### Pandas 활용 1. 일부분만 관찰
- head(N) : 앞에서 N개의 데이터를 참ㄷ조

In [21]:
covid.head(5)

,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region
0,Afghanistan,36263,1269,25198,9796,106,10,18,3.50,69.49,5.04,35526,737,2.07,Eastern Mediterranean
1,Albania,4880,144,2745,1991,117,6,63,2.95,56.25,5.25,4171,709,17.00,Europe
2,Algeria,27973,1163,18837,7973,616,8,749,4.16,67.34,6.17,23691,4282,18.07,Africa
3,Andorra,907,52,803,52,10,0,0,5.73,88.53,6.48,884,23,2.60,Europe
4,Angola,950,41,242,667,18,1,0,4.32,25.47,16.94,749,201,26.84,Africa


- tail(N) : 아래서 N개 만큼의 데이터 확인

In [22]:
covid.tail(5)

,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region
182,West Bank and Gaza,10621,78,3752,6791,152,2,0,0.73,35.33,2.08,8916,1705,19.12,Eastern Mediterranean
183,Western Sahara,10,1,8,1,0,0,0,10.00,80.00,12.50,10,0,0.00,Africa
184,Yemen,1691,483,833,375,10,4,36,28.56,49.26,57.98,1619,72,4.45,Eastern Mediterranean
185,Zambia,4552,140,2815,1597,71,1,465,3.08,61.84,4.97,3326,1226,36.86,Africa
186,Zimbabwe,2704,36,542,2126,192,2,24,1.33,20.04,6.64,1713,991,57.85,Africa


#### Pandas 활용 2. 데이터 접근하기
- df['columns'] or df.columns

In [23]:
covid['Active']

0      9796
1      1991
2      7973
3        52
4       667
       ... 
182    6791
183       1
184     375
185    1597
186    2126
Name: Active, Length: 187, dtype: int64

In [24]:
covid.Active

0      9796
1      1991
2      7973
3        52
4       667
       ... 
182    6791
183       1
184     375
185    1597
186    2126
Name: Active, Length: 187, dtype: int64

#### 꿀팁
- Dataframe의 각 columns은 'Series'다
- 가져온 데이터를 DataFrame으로 만드는 방법

In [25]:
covid['Confirmed'][0:3]

0    36263
1     4880
2    27973
Name: Confirmed, dtype: int64

#### Pandas 활용 3 '조건'을 이용하여 데이터 접근

In [26]:
# 신규 확진자가 100명이 넘는 나라를 찾기
covid[covid['New cases'] > 100]

,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region
0,Afghanistan,36263,1269,25198,9796,106,10,18,3.50,69.49,5.04,35526,737,2.07,Eastern Mediterranean
1,Albania,4880,144,2745,1991,117,6,63,2.95,56.25,5.25,4171,709,17.00,Europe
2,Algeria,27973,1163,18837,7973,616,8,749,4.16,67.34,6.17,23691,4282,18.07,Africa
6,Argentina,167416,3059,72575,91782,4890,120,2057,1.83,43.35,4.21,130774,36642,28.02,Americas
8,Australia,15303,167,9311,5825,368,6,137,1.09,60.84,1.79,12428,2875,23.13,Western Pacific
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,United Kingdom,301708,45844,1437,254427,688,7,3,15.19,0.48,3190.26,296944,4764,1.60,Europe
179,Uzbekistan,21209,121,11674,9414,678,5,569,0.57,55.04,1.04,17149,4060,23.67,Europe
180,Venezuela,15988,146,9959,5883,525,4,213,0.91,62.29,1.47,12334,3654,29.63,Americas
182,West Bank and Gaza,10621,78,3752,6791,152,2,0,0.73,35.33,2.08,8916,1705,19.12,Eastern Mediterranean


In [27]:
#  WHO 지역이 동남아시아인 나라 찾기
# unique () 범주형 자료에서 데이터를 유니크하게 보여준다
covid['WHO Region'].unique()

array(['Eastern Mediterranean', 'Europe', 'Africa', 'Americas',
       'Western Pacific', 'South-East Asia'], dtype=object)

In [28]:
covid[covid['WHO Region'] == 'South-East Asia'].head()

,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region
13,Bangladesh,226225,2965,125683,97577,2772,37,1801,1.31,55.56,2.36,207453,18772,9.05,South-East Asia
19,Bhutan,99,0,86,13,4,0,1,0.00,86.87,0.00,90,9,10.00,South-East Asia
27,Burma,350,6,292,52,0,0,2,1.71,83.43,2.05,341,9,2.64,South-East Asia
79,India,1480073,33408,951166,495499,44457,637,33598,2.26,64.26,3.51,1155338,324735,28.11,South-East Asia
80,Indonesia,100303,4838,58173,37292,1525,57,1518,4.82,58.00,8.32,88214,12089,13.70,South-East Asia


#### Pandas 활용4. 행을 기준으로 데이터 접근하기

In [29]:
# 예시 데이터
books_dict = {'Available' :[True,True,False], 'location' : [102,215,323], 'Genre':['Programming','Physics','Math']}
books_df = pd.DataFrame(books_dict, index=['버그란 무엇인가','두근두근 물리학','미분해줘 홈즈'])
books_df

,Available,location,Genre
버그란 무엇인가,True,102,Programming
두근두근 물리학,True,215,Physics
미분해줘 홈즈,False,323,Math


#### 인덱스를 이용해서 가져오기
- .loc[row,col]

In [30]:
books_df.loc['버그란 무엇인가']

Available           True
location             102
Genre        Programming
Name: 버그란 무엇인가, dtype: object

In [31]:
# 미분해줘 홈즈 책이 대출 가능한지?
books_df.loc['미분해줘 홈즈', 'Available']

False

#### 숫자 인덱스를 이용하여 가져오기
- .`iloc[rowidx,colidx]`

In [32]:
# 인덱스 0행 1열
books_df.iloc[0,1]

102

In [33]:
# 인덱스 1행의 인덱스 0~1 가져오기
books_df.iloc[1,:2]

Available    True
location      215
Name: 두근두근 물리학, dtype: object

#### Pandas 활용 5. Groupby
- Split : 특정한 '기준'을 바탕으로 DF를 분할
- Apply : 통계함수 - sum().mean().median() 을 적용해서 각 데이터를 압축
- Combine:Apply된 결과를 바탕으로 새로운 Series를 생성 (group_key : applied_val)

In [34]:
covid.head()

,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region
0,Afghanistan,36263,1269,25198,9796,106,10,18,3.50,69.49,5.04,35526,737,2.07,Eastern Mediterranean
1,Albania,4880,144,2745,1991,117,6,63,2.95,56.25,5.25,4171,709,17.00,Europe
2,Algeria,27973,1163,18837,7973,616,8,749,4.16,67.34,6.17,23691,4282,18.07,Africa
3,Andorra,907,52,803,52,10,0,0,5.73,88.53,6.48,884,23,2.60,Europe
4,Angola,950,41,242,667,18,1,0,4.32,25.47,16.94,749,201,26.84,Africa


In [35]:
# WHO Region 별 확진자수

# 1.covid에서 확진자 수 col만을 추출한다
# 2. 이를 covid의 WHO Region을 기준으로 groupby한다.
covid_by_region = covid['Confirmed'].groupby(by=covid['WHO Region'])
covid_by_region

In [36]:
# 총 확진자 수
covid_by_region.sum()

WHO Region
Africa                    723207
Americas                 8839286
Eastern Mediterranean    1490744
Europe                   3299523
South-East Asia          1835297
Western Pacific           292428
Name: Confirmed, dtype: int64

In [37]:
# 국가당 감염자 수
covid_by_region.mean() # sum / 국가수

WHO Region
Africa                    15066.812500
Americas                 252551.028571
Eastern Mediterranean     67761.090909
Europe                    58920.053571
South-East Asia          183529.700000
Western Pacific           18276.750000
Name: Confirmed, dtype: float64

## Mission:
### 1. covid 데이터에서 100 case 대비 사망률(`Deaths / 100 Cases`)이 가장 높은 국가는?

In [38]:
highest_death = max(covid['Deaths / 100 Cases'].unique())
covid[covid['Deaths / 100 Cases'] == highest_death]['Country/Region']

184    Yemen
Name: Country/Region, dtype: object

### 2. covid 데이터에서 신규 확진자가 없는 나라 중 WHO Region이 'Europe'를 모두 출력하면?  
Hint : 한 줄에 동시에 두가지 조건을 Apply하는 경우 Warning이 발생할 수 있습니다.

In [39]:
covid_no_case = covid[covid['New cases'] == 0]
covid_no_case[covid_no_case['WHO Region'] == 'Europe']

,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region
56,Estonia,2034,69,1923,42,0,0,1,3.39,94.54,3.59,2021,13,0.64,Europe
75,Holy See,12,0,12,0,0,0,0,0.00,100.00,0.00,12,0,0.00,Europe
95,Latvia,1219,31,1045,143,0,0,0,2.54,85.73,2.97,1192,27,2.27,Europe
100,Liechtenstein,86,1,81,4,0,0,0,1.16,94.19,1.23,86,0,0.00,Europe
113,Monaco,116,4,104,8,0,0,0,3.45,89.66,3.85,109,7,6.42,Europe
143,San Marino,699,42,657,0,0,0,0,6.01,93.99,6.39,699,0,0.00,Europe
157,Spain,272421,28432,150376,93613,0,0,0,10.44,55.20,18.91,264836,7585,2.86,Europe


### 3. 다음 [데이터](https://www.kaggle.com/neuromusic/avocado-prices)를 이용해 각 Region별로 아보카도가 가장 비싼 평균가격(AveragePrice)을 출력하면?

In [52]:
avocado = pd.read_csv('/content/avocado.csv')
avocado.groupby('region')['AveragePrice'].max()

region
Albany                 2.13
Atlanta                2.75
BaltimoreWashington    2.28
Boise                  2.79
Boston                 2.19
BuffaloRochester       2.57
California             2.58
Charlotte              2.83
Chicago                2.30
CincinnatiDayton       2.20
Columbus               2.22
DallasFtWorth          1.90
Denver                 2.16
Detroit                2.08
GrandRapids            2.73
GreatLakes             1.98
HarrisburgScranton     2.27
HartfordSpringfield    2.68
Houston                1.92
Indianapolis           2.10
Jacksonville           2.99
LasVegas               3.03
LosAngeles             2.44
Louisville             2.29
MiamiFtLauderdale      3.05
Midsouth               2.17
Nashville              2.24
NewOrleansMobile       2.32
NewYork                2.65
Northeast              2.31
NorthernNewEngland     1.96
Orlando                2.87
Philadelphia           2.45
PhoenixTucson          2.62
Pittsburgh             1.83
Plains       